In [1]:
#importing required libraries

#image precessing
import cv2
#file handling
import os
#obvious
import numpy as np
import pandas as pd
#library to know the progress of for loops
from tqdm import tqdm
# from matplotlib import pyplot as plt
from keras.utils import np_utils
from sklearn.utils import shuffle

Using TensorFlow backend.


In [2]:
#the path where the required files are
x = []
y = []
IMG_SIZE = 64
base_dir = 'Gestures'
for i in tqdm(os.listdir(base_dir)):
    path = os.path.join(base_dir,i)
    for j in os.listdir(path):
        img = cv2.imread(os.path.join(path,j),0)
#         print(img.shape)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        img = img/255
        if img is not None:
            x.append(img)
            y.append(i)

100%|██████████| 6/6 [00:02<00:00,  2.78it/s]


In [3]:
x = np.array(x)
y = np.array(y)

In [4]:
x.shape,y.shape

((7093, 64, 64), (7093,))

In [5]:
y = np_utils.to_categorical(y)
print(y.shape)

(7093, 6)


In [6]:
x,y = shuffle(x,y,random_state = 0)

In [7]:
x = x.reshape(x.shape[0],IMG_SIZE,IMG_SIZE,1)
x.shape,y.shape

((7093, 64, 64, 1), (7093, 6))

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

In [9]:
#multi classification model
nClasses = 6

In [10]:
#constructing the model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(IMG_SIZE,IMG_SIZE,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
 
model.add(Flatten())
model.add(Dense(nClasses, activation='softmax'))

In [11]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 64, 64, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)             0         
__________

In [12]:
model.fit(x,y,epochs =5,batch_size = 100)

Epoch 1/5
7093/7093 [==============================] - 34s 5ms/step - loss: 0.3790 - acc: 0.8711
Epoch 2/5
7093/7093 [==============================] - 33s 5ms/step - loss: 0.0388 - acc: 0.9904
Epoch 3/5
7093/7093 [==============================] - 33s 5ms/step - loss: 0.0103 - acc: 0.9986
Epoch 4/5
7093/7093 [==============================] - 33s 5ms/step - loss: 0.0051 - acc: 0.9990
Epoch 5/5
7093/7093 [==============================] - 33s 5ms/step - loss: 0.0027 - acc: 0.9999


In [13]:
loss2, acc2 = model.evaluate(x,y)

7093/7093 [==============================] - 9s 1ms/step


In [14]:
acc2

1.0

In [15]:
model.save("Gesture-model-saved.h5")